In [1]:
import h2o
import pandas as pd

from h2o.automl import H2OAutoML

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,21 hours 3 mins
H2O cluster timezone:,Asia/Seoul
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.5
H2O cluster version age:,13 days
H2O cluster name:,H2O_from_python_root_rp8ljb
H2O cluster total nodes:,1
H2O cluster free memory:,13.13 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [3]:
data_set = pd.read_csv('rawToRemake_seq_1days_2up_5step_output.csv')
data_set.head(5)

,date,product_no,month,day,w_day,season,price,quantity_0,clicks_0,quantity_5,quantity_4,quantity_3,quantity_2,quantity_1,clicks_5,clicks_4,clicks_3,clicks_2,clicks_1,output
0,2017-01-06 0:00,97616,1,6,4,1,13800,0,3,0,2,0,0,0,0,3,3,3,5,0
1,2017-01-07 0:00,97616,1,7,5,1,13800,0,1,2,0,0,0,0,3,3,3,5,3,0
2,2017-03-03 0:00,97616,3,3,4,2,13800,1,14,0,0,0,0,0,14,25,12,11,11,0
3,2017-03-04 0:00,97616,3,4,5,2,13800,0,13,0,0,0,0,1,25,12,11,11,14,0
4,2017-03-05 0:00,97616,3,5,6,2,13800,0,9,0,0,0,1,0,12,11,11,14,13,0


In [4]:
del data_set['date']
del data_set['product_no']
data_set.head(5)

,month,day,w_day,season,price,quantity_0,clicks_0,quantity_5,quantity_4,quantity_3,quantity_2,quantity_1,clicks_5,clicks_4,clicks_3,clicks_2,clicks_1,output
0,1,6,4,1,13800,0,3,0,2,0,0,0,0,3,3,3,5,0
1,1,7,5,1,13800,0,1,2,0,0,0,0,3,3,3,5,3,0
2,3,3,4,2,13800,1,14,0,0,0,0,0,14,25,12,11,11,0
3,3,4,5,2,13800,0,13,0,0,0,0,1,25,12,11,11,14,0
4,3,5,6,2,13800,0,9,0,0,0,1,0,12,11,11,14,13,0


In [5]:
data_from_df = h2o.H2OFrame(data_set)
data_from_df.head(5)

/home/notebook/jhko01/AIMD_PLATFORM/lib/python2.7/site-packages/h2o/utils/shared_utils.py:177: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


month,day,w_day,season,price,quantity_0,clicks_0,quantity_5,quantity_4,quantity_3,quantity_2,quantity_1,clicks_5,clicks_4,clicks_3,clicks_2,clicks_1,output
1,6,4,1,13800,0,3,0,2,0,0,0,0,3,3,3,5,0
1,7,5,1,13800,0,1,2,0,0,0,0,3,3,3,5,3,0
3,3,4,2,13800,1,14,0,0,0,0,0,14,25,12,11,11,0
3,4,5,2,13800,0,13,0,0,0,0,1,25,12,11,11,14,0
3,5,6,2,13800,0,9,0,0,0,1,0,12,11,11,14,13,0


In [6]:
data_split = data_from_df.split_frame(ratios = [0.8], seed = 1234)

data_train = data_split[0]
data_test = data_split[1]

In [7]:
x = data_train.columns
y = 'output'
x.remove(y)

In [8]:
x

[u'month',
 u'day',
 u'w_day',
 u'season',
 u'price',
 u'quantity_0',
 u'clicks_0',
 u'quantity_5',
 u'quantity_4',
 u'quantity_3',
 u'quantity_2',
 u'quantity_1',
 u'clicks_5',
 u'clicks_4',
 u'clicks_3',
 u'clicks_2',
 u'clicks_1']

In [17]:
data_train[y] = data_train[y].asfactor()
data_test[y] = data_test[y].asfactor()
print(data_train[y].table())
print(data_test[y].table())

output,Count
0,131604
1,103607


output,Count
0,32797
1,25716


In [10]:
aml = H2OAutoML(max_runtime_secs = 30)
aml.train(x = x, y = y,
          training_frame = data_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [11]:
aml.leaderboard

model_id,auc,logloss,mean_per_class_error,rmse,mse
DRF_0_AutoML_20180822_151404,0.845583,0.478794,0.240649,0.39446,0.155599


In [12]:
aml.leader

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_0_AutoML_20180822_151404


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.158621750957
RMSE: 0.398273462532
LogLoss: 0.506345660425
Mean Per-Class Error: 0.239559607313
AUC: 0.839392892105
Gini: 0.678785784209
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.385429404652: 


,0,1,Error,Rate
0,78870.0,26215.0,0.2495,(26215.0/105085.0)
1,19593.0,63240.0,0.2365,(19593.0/82833.0)
Total,98463.0,89455.0,0.2438,(45808.0/187918.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3854294,0.7341196,220.0
max f2,0.1593215,0.8190033,324.0
max f0point5,0.6028932,0.7660516,133.0
max accuracy,0.4944895,0.7691600,174.0
max precision,0.9999843,0.9878936,0.0
max recall,0.0002576,1.0,399.0
max specificity,0.9999843,0.9984489,0.0
max absolute_mcc,0.5168394,0.5291689,165.0
max min_per_class_accuracy,0.3925970,0.7568723,217.0
max mean_per_class_accuracy,0.4554901,0.7604404,190.0


Gains/Lift Table: Avg response rate: 44.08 %, avg score: 44.02 %



...

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.7572185,0.0039235,0.7623723,0.7627714,0.757583,0.7558737,0.7474922
auc,0.8456153,0.0018631,0.8485928,0.8477569,0.8456054,0.8450860,0.8410352
err,0.2427815,0.0039235,0.2376277,0.2372286,0.242417,0.2441263,0.2525078
err_count,9124.6,147.3897,8931.0,8916.0,9111.0,9175.0,9490.0
f0point5,0.7166307,0.0057070,0.7229443,0.7253369,0.7182504,0.7141836,0.7024381
f1,0.7390633,0.0027283,0.7425854,0.7425056,0.7392312,0.7390130,0.7319815
f2,0.763011,0.0012969,0.7633235,0.7605069,0.7614746,0.7656309,0.764119
lift_top_group,2.259007,0.0056243,2.259239,2.250159,2.2602022,2.252494,2.2729404
logloss,0.4787945,0.0033985,0.4762723,0.4760507,0.4728263,0.4856123,0.4832107
max_per_class_error,0.2606134,0.0095646,0.2497859,0.2453470,0.2578583,0.2666476,0.2834280


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-08-22 15:15:17,1 min 12.939 sec,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-08-22 15:15:17,1 min 13.158 sec,1.0,0.5037612,6.4455357,0.7062712,1.6599976,0.2945307,0.5027719,6.4709750,0.7085279,1.6524937,0.2967247
,2018-08-22 15:15:17,1 min 13.422 sec,2.0,0.4946421,5.6880891,0.7201962,1.6910765,0.2960547,0.4522812,2.6330573,0.7675059,1.9207254,0.2966612
,2018-08-22 15:15:17,1 min 13.715 sec,3.0,0.4851260,4.9508036,0.7311967,1.7313043,0.3019760,0.4325534,1.4575679,0.7936802,2.0488358,0.3040408
,2018-08-22 15:15:18,1 min 14.025 sec,4.0,0.4735502,4.1405887,0.7432152,1.7805989,0.2997079,0.4221886,0.9900448,0.8067665,2.1153585,0.2653247
,2018-08-22 15:15:18,1 min 14.574 sec,5.0,0.4643623,3.5106549,0.7538970,1.8259325,0.2976529,0.4156287,0.7946237,0.8151791,2.1539982,0.2686656
,2018-08-22 15:15:19,1 min 14.907 sec,6.0,0.4561112,2.9780281,0.7639421,1.8651982,0.2970707,0.4112366,0.6943536,0.8209748,2.1787793,0.2777578
,2018-08-22 15:15:19,1 min 15.264 sec,7.0,0.4487382,2.5176855,0.7728213,1.9066243,0.2957933,0.4080198,0.6432615,0.8253042,2.1934873,0.2651767
,2018-08-22 15:15:19,1 min 15.655 sec,8.0,0.4428983,2.1631045,0.7799960,1.9427658,0.2944255,0.4058221,0.6095773,0.8283340,2.2036795,0.2613917
,2018-08-22 15:15:20,1 min 16.000 sec,9.0,0.4374143,1.8614341,0.7867918,1.9747817,0.2923225,0.4039691,0.5819291,0.8309258,2.2138858,0.2529338


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
clicks_0,187478.7031250,1.0,0.1487741
clicks_1,144885.1562500,0.7728086,0.1149739
quantity_0,118618.8828120,0.6327059,0.0941303
clicks_2,115462.4531250,0.6158697,0.0916255
quantity_1,94995.8046875,0.5067018,0.0753841
clicks_3,85524.3593750,0.4561817,0.0678680
clicks_4,75702.4140625,0.4037921,0.0600738
quantity_2,64211.625,0.3425009,0.0509553
clicks_5,63754.3164062,0.3400616,0.0505924
price,57215.0546875,0.3051816,0.0454031


In [13]:
model_path = h2o.save_model(model=aml.leader, path="h2o_ex/cafe24_h2o_model", force=True)

In [14]:
aml.leader.predict(data_test)

drf prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
0,0.90381,0.0961896
0,0.709256,0.290744
0,0.643863,0.356137
0,0.767357,0.232643
0,0.769069,0.230931
0,0.613755,0.386245
0,0.787017,0.212983
0,0.794785,0.205215
0,0.914431,0.0855694
0,0.949994,0.0500059
